In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 1.2 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=389ed3374c2ab1843345ef0ceeb7c3e7f527aaa5d4181143bcf07d60adda2106
  Stored in directory: /tmp/pip-ephem-wheel-cache-32l0fh8w/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
pip install torchtext

     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:39tcmalloc: large alloc 1147494400 bytes == 0x55b597056000 @  0x7f58d3a4d615 0x55b55de7617c 0x55b55df5647a 0x55b55de78f9d 0x55b55df6ad4d 0x55b55deecec8 0x55b55dee7a2e 0x55b55de7a88a 0x55b55deecd30 0x55b55dee7a2e 0x55b55de7a88a 0x55b55dee9719 0x55b55df6bb76 0x55b55dee8d95 0x55b55df6bb76 0x55b55dee8d95 0x55b55df6bb76 0x55b55dee8d95 0x55b55de7ace9 0x55b55debe579 0x55b55de79902 0x55b55deecc4d 0x55b55dee7a2e 0x55b55de7a88a 0x55b55dee9719 0x55b55dee7a2e 0x55b55de7a88a 0x55b55dee88f6 0x55b55de7a7aa 0x55b55dee8b4f 0x55b55dee7a2e
     |████████████████████████████████| 881.9 MB 15 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0


In [ ]:
from torchtext.legacy.data import Field, LabelField
from torchtext.legacy.data import TabularDataset
from torchtext.legacy.data import Iterator, BucketIterator

In [ ]:
import torch
import torchtext
# from torchtext.data import Field, LabelField # For torch<=0.8.0, the importing of functions should be `from torchtext.data`
# from torchtext.data import TabularDataset
# from torchtext.data import Iterator, BucketIterator
import spacy
import en_core_web_lg
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
from tqdm import tqdm, trange
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix

manual_seed = 77
torch.manual_seed(manual_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
n_gpu = torch.cuda.device_count()
if n_gpu > 0:
    torch.cuda.manual_seed(manual_seed)

cpu


In [ ]:
spacy_en = en_core_web_lg.load()

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

stopwords = spacy_en.Defaults.stop_words    

In [92]:
TEXT = Field(sequential=True, tokenize=tokenize_en, lower=True)
LABEL = Field(sequential=False, unk_token = None)

train, val = TabularDataset.splits(
               path="./drive/My Drive/CNN/", # the root directory where the data lies
               train='train.tsv', validation="dev.tsv", # file names
               format='tsv',
               skip_header=False, # if your tsv file has a header, make sure to pass this to ensure it doesn't get proceesed as data!
               fields=[(None, None),('label', LABEL), ('tweet', TEXT)])

In [93]:
test = TabularDataset.splits(
               path="/content/drive/MyDrive/CNN/", # the root directory where the data lies
               test="test_masks.tsv", # file names
               format='tsv',
               skip_header=True, # if your tsv file has a header, make sure to pass this to ensure it doesn't get proceesed as data!
               fields=[(None, None), ('tweet', TEXT), ('label', LABEL)])

In [94]:
test = test[0]

In [ ]:
# import torchtext.vocab as vocab
# custom_embeddings = vocab.Vectors(name = root_path + 'glove.twitter.27B.200d.txt',
#                                   cache = 'custom_embeddings',
#                                   unk_init = torch.Tensor.normal_)
from zipfile import ZipFile
  
# specifying the zip file name
file_name = "./drive/My Drive/CNN/emb.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

File Name                                             Modified             Size
glove.twitter.27B.100d.txt                     2015-12-22 16:04:54   1021671926
glove.twitter.27B.200d.txt                     2015-12-22 16:04:54   2057595650
glove.twitter.27B.25d.txt                      2015-12-22 16:04:54    257699930
glove.twitter.27B.50d.txt                      2015-12-22 16:04:54    510889212
Extracting all the files now...
Done!


In [95]:
# pre_trained_vector_type = '/content/drive/MyDrive/CNN/glove.twitter.27B.200d.txt'
import torchtext.vocab as vocab
loaded_vectors = torchtext.vocab.Vectors('/content/drive/MyDrive/CNN/glove.twitter.27B.200d.txt')
# fine_trained_vectors = vocab.Vectors('/content/drive/MyDrive/CNN/glove.twitter.27B.200d.txt')
# TEXT.build_vocab(train, vectors=fine_trained_vectors)
TEXT.build_vocab(train, vectors=loaded_vectors, max_size=len(loaded_vectors.stoi))
TEXT.vocab.set_vectors(stoi=loaded_vectors.stoi, vectors=loaded_vectors.vectors, dim=loaded_vectors.dim)
LABEL.build_vocab(train)
print("Vocabulary size of TEXT:",len(TEXT.vocab.stoi))
print("Vocabulary size of LABEL:",len(LABEL.vocab.stoi))

Vocabulary size of TEXT: 37888
Vocabulary size of LABEL: 3


In [96]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import random
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train, val, test), 
    batch_size = 32,
    sort_key=lambda x: len(x.tweet), 
    sort=True,
    sort_within_batch=True,
    device = device)

In [97]:
len(test_iter)

3

In [ ]:
class CNN_Text(nn.Module):
    def __init__(self, vocabulary_size, embedding_dim, output_size, kernel_num, region_sizes, dropout):
        '''
        vocabulary_size: vocabulary size
        embedding_dim: word embedding size
        output_size: number of classes in prediction
        kernel_num: number of kernels (number of output channels of convolutional layers)
        region_sizes: height of kernels of convolutional layers
        dropout: dropout rate
        '''
        super(CNN_Text, self).__init__()
        # the size of input channel is 1.
        Ci = 1
        
        # word embedding layer
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size, embedding_dim = embedding_dim )
        
        # convolution with kernels
        self.convolution_layers = nn.ModuleList([nn.Conv2d(in_channels = Ci, out_channels = kernel_num, kernel_size = (K, embedding_dim)) for K in region_sizes])
        
        # a dropout layer
        self.dropout = nn.Dropout(dropout) 
        
        # fully connected layer
        self.fc = nn.Linear(len(kernel_sizes) * kernel_num, output_size)

    def forward(self, x):
        # input x  [sequence length, batch size]
        
        input_embeddings = self.embeddings(x)  
        # (batch size, word_sequence, embedding_dim) word embedding

        input_embeddings = input_embeddings.permute(1,0,2)
        input_embeddings = input_embeddings.unsqueeze(1)
        #  [batch size, number of channel is one, sequence length, embeeding size]

        # convolutional layers
        convolute_outputs = [F.relu(conv(input_embeddings)).squeeze(3) for conv in self.convolution_layers]  
        
        # to get the maximum value of filtered tensor
        max_pooling_outputs = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in convolute_outputs] 
        
        concat_list = torch.cat(max_pooling_outputs, 1) # concatenate representations
        
        drop_output = self.dropout(concat_list)  # add drop layer
        
        fc1_output = self.fc(drop_output)  # get the fc1 using a fully connected layer
        
        final_output = F.softmax(fc1_output,dim=1)
        
        return final_output

In [ ]:
class CNN_BiLSTM(nn.Module):

    def __init__(self, args):
        super(CNN_BiLSTM, self).__init__()
        self.args = args
        self.hidden_dim = args.lstm_hidden_dim
        self.num_layers = args.lstm_num_layers
        V = args.embed_num
        D = args.embed_dim
        C = args.class_num
        self.C = C
        Ci = 1
        Co = args.kernel_num
        Ks = args.kernel_sizes
        self.embed = nn.Embedding(V, D, padding_idx=args.paddingId)
        # pretrained  embedding
        if args.word_Embedding:
            self.embed.weight.data.copy_(args.pretrained_weight)

        # CNN
        self.convs1 = [nn.Conv2d(Ci, Co, (K, D), padding=(K//2, 0), stride=1) for K in Ks]
        print(self.convs1)
        # for cnn cuda
        # if self.args.cuda is True:
        #     for conv in self.convs1:
        #         conv = conv.cuda()

        # BiLSTM
        self.bilstm = nn.LSTM(D, self.hidden_dim, num_layers=self.num_layers, dropout=args.dropout, bidirectional=True, bias=True)

        # linear
        L = len(Ks) * Co + self.hidden_dim * 2
        self.hidden2label1 = nn.Linear(L, L // 2)
        self.hidden2label2 = nn.Linear(L // 2, C)

        # dropout
        self.dropout = nn.Dropout(args.dropout)

    def forward(self, x):
        embed = self.embed(x)

        # CNN
        cnn_x = embed
        cnn_x = torch.transpose(cnn_x, 0, 1)
        cnn_x = cnn_x.unsqueeze(1)
        cnn_x = [F.relu(conv(cnn_x)).squeeze(3) for conv in self.convs1]  # [(N,Co,W), ...]*len(Ks)
        cnn_x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in cnn_x]  # [(N,Co), ...]*len(Ks)
        cnn_x = torch.cat(cnn_x, 1)
        cnn_x = self.dropout(cnn_x)

        # BiLSTM
        bilstm_x = embed.view(len(x), embed.size(1), -1)
        bilstm_out, _ = self.bilstm(bilstm_x)
        bilstm_out = torch.transpose(bilstm_out, 0, 1)
        bilstm_out = torch.transpose(bilstm_out, 1, 2)
        bilstm_out = F.max_pool1d(bilstm_out, bilstm_out.size(2)).squeeze(2)
        # bilstm_out = F.tanh(bilstm_out)

        # CNN and BiLSTM CAT
        cnn_x = torch.transpose(cnn_x, 0, 1)
        bilstm_out = torch.transpose(bilstm_out, 0, 1)
        cnn_bilstm_out = torch.cat((cnn_x, bilstm_out), 0)
        cnn_bilstm_out = torch.transpose(cnn_bilstm_out, 0, 1)

        # linear
        cnn_bilstm_out = self.hidden2label1(F.tanh(cnn_bilstm_out))
        cnn_bilstm_out = self.hidden2label2(F.tanh(cnn_bilstm_out))

        # output
        logit = cnn_bilstm_out
        return logit

In [98]:
# Hyper Parameters

# the vocabulary size
vocabulary_size = len(TEXT.vocab.stoi) 
hidden_dim = 512
# Dimension of word embedding is 300. Namely, each word is expressed by a vector that has 300 dimensions.
embedding_dim = 200 
num_layers = 2
# region size as 2, 3, and 4
kernel_sizes = [2,3,4] 

# the number of kernel in each region size
kernels_num = 32 

# The dropout rate is set to be 0.5.
dropout = 0.2

# The output size of labels.
output_size = 3

# learning rate is set to be 0.01.
lr = 0.001        
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
# The number of iteration is set to be 5.
num_epoch = 20 

class CNNArgs:
  def __init__(self, hidden_dim, num_layers, vocabulary_size, embedding_dim, output_size, kernels_num, kernel_sizes, dropout) -> None:
      self.lstm_hidden_dim = hidden_dim
      self.lstm_num_layers = num_layers
      self.embed_num = vocabulary_size 
      self.embed_dim = embedding_dim
      self.class_num = output_size
      self.kernel_num = kernels_num
      self.kernel_sizes = kernel_sizes
      self.dropout = dropout
      self.paddingId = None
      self.word_Embedding = None
      self.cuda = True


args_for_cnn = CNNArgs(hidden_dim, num_layers, vocabulary_size, embedding_dim, output_size, kernels_num, kernel_sizes, dropout)


# employ class CNN_Text and assign to cnn
model = CNN_BiLSTM(args_for_cnn).to(device)
pretrained_embeddings = TEXT.vocab.vectors

model.embed.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embed.weight.data[UNK_IDX] = torch.zeros(embedding_dim)
model.embed.weight.data[PAD_IDX] = torch.zeros(embedding_dim)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)   # define a optimizer for backpropagation
# loss_func = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

 

criterion = nn.CrossEntropyLoss()


[Conv2d(1, 32, kernel_size=(2, 200), stride=(1, 1), padding=(1, 0)), Conv2d(1, 32, kernel_size=(3, 200), stride=(1, 1), padding=(1, 0)), Conv2d(1, 32, kernel_size=(4, 200), stride=(1, 1), padding=(2, 0))]


In [99]:
model.load_state_dict(torch.load('./drive/My Drive/CNN/CNN_LSTM_1.pt')['state_dict'])

<All keys matched successfully>

In [ ]:
# for i in train_iter:
#   print(i)

In [ ]:
test_loss, test_acc, test_f1 = evaluate(model, val_iter, criterion)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [100]:
test_loss, test_acc, test_f1 = evaluate(model, test_iter, criterion)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [101]:
# evaluation on vaccines test set 
# (0.8434282392263412, 0.6831683168316832, 0.7058374249863612)
# evaluation on masks test set
# (1.070494016011556, 0.45555555555555555, 0.4294701675136458)
test_loss, test_acc, test_f1

(1.070494016011556, 0.45555555555555555, 0.4294701675136458)

In [ ]:
# CNN + BiLSTM with weights
optimizer = torch.optim.Adam(model.parameters(), lr=lr)   # define a optimizer for backpropagation
loss_func = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

 

criterion = nn.CrossEntropyLoss()
MAX_EPOCH = 15
total_step = len(train_iter)
loss_list = []
acc_list = []

for epoch in trange(MAX_EPOCH, desc="Epoch"):
    train_loss = train(model, train_iter, optimizer, criterion)  
    val_loss, val_acc, val_f1 = evaluate(model, val_iter, criterion)

    # Create checkpoint at end of each epoch
    state_dict_model = model.state_dict() 
    state = {
        'epoch': epoch,
        'state_dict': state_dict_model,
        'optimizer': optimizer.state_dict()
        }

    torch.save(state, "./drive/My Drive/CNN/CNN_LSTM_"+str(epoch+1)+".pt")

    print('\n Epoch [{}/{}], Train Loss: {:.4f}, Validation Loss: {:.4f}, Validation Accuracy: {:.4f}, Validation F1: {:.4f}'.format(epoch+1, MAX_EPOCH, train_loss, val_loss, val_acc, val_f1))


Epoch:   0%|          | 0/15 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
Epoch:   7%|▋         | 1/15 [11:11<2:36:40, 671.46s/it]


 Epoch [1/15], Train Loss: 0.2706, Validation Loss: 0.9916, Validation Accuracy: 0.5591, Validation F1: 0.5499


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
Epoch:  13%|█▎        | 2/15 [23:28<2:33:53, 710.25s/it]


 Epoch [2/15], Train Loss: 0.1426, Validation Loss: 1.4493, Validation Accuracy: 0.5388, Validation F1: 0.5145


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
Epoch:  20%|██        | 3/15 [35:45<2:24:29, 722.45s/it]

...


 Epoch [13/15], Train Loss: 0.0365, Validation Loss: 2.0675, Validation Accuracy: 0.5340, Validation F1: 0.5133


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
Epoch:  93%|█████████▎| 14/15 [2:44:38<10:56, 656.56s/it]


 Epoch [14/15], Train Loss: 0.0216, Validation Loss: 2.3923, Validation Accuracy: 0.5390, Validation F1: 0.5092


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
Epoch: 100%|██████████| 15/15 [2:53:31<00:00, 694.08s/it]


 Epoch [15/15], Train Loss: 0.0173, Validation Loss: 2.7553, Validation Accuracy: 0.5351, Validation F1: 0.5084


In [ ]:
# Hyper Parameters

# the vocabulary size
vocabulary_size = len(TEXT.vocab.stoi) 

# Dimension of word embedding is 300. Namely, each word is expressed by a vector that has 300 dimensions.
embedding_dim = 200 

# region size as 2, 3, and 4
kernel_sizes = [2,3,4] 

# the number of kernel in each region size
kernels_num = 32 

# The dropout rate is set to be 0.5.
dropout = 0.2

# The output size of labels.
output_size = 3

# learning rate is set to be 0.01.
lr = 0.001        

# The number of iteration is set to be 5.
num_epoch = 20  

# employ class CNN_Text and assign to cnn
model = CNN_Text(vocabulary_size, embedding_dim, output_size, kernels_num, kernel_sizes, dropout).to(device)

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        batch_input, labels = batch.tweet, batch.label
        batch_input = batch_input.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(batch_input)
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.cpu().item()

    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    all_pred=[]
    all_label = []
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            batch_input, labels = batch.tweet, batch.label
            batch_input = batch_input.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(batch_input)

            loss = criterion(outputs, labels)

            epoch_loss += loss.cpu().item()

            # identify the predicted class for each example in the batch
            probabilities, predicted = torch.max(outputs.cpu().data, 1)
            # put all the true labels and predictions to two lists
            all_pred.extend(predicted)
            all_label.extend(labels.cpu())
    
    accuracy = accuracy_score(all_label, all_pred)
    f1score = f1_score(all_label, all_pred, average='macro') 
    return epoch_loss / len(iterator), accuracy, f1score

In [ ]:
# Train the model CNN + BiLSLM

optimizer = torch.optim.Adam(model.parameters(), lr=lr)   # define a optimizer for backpropagation
loss_func = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

 

criterion = nn.CrossEntropyLoss()
MAX_EPOCH = 15
total_step = len(train_iter)
loss_list = []
acc_list = []

for epoch in trange(MAX_EPOCH, desc="Epoch"):
    train_loss = train(model, train_iter, optimizer, criterion)  
    val_loss, val_acc, val_f1 = evaluate(model, val_iter, criterion)

    # Create checkpoint at end of each epoch
    state_dict_model = model.state_dict() 
    state = {
        'epoch': epoch,
        'state_dict': state_dict_model,
        'optimizer': optimizer.state_dict()
        }

    # torch.save(state, "./drive/My Drive/Colab Notebooks/ckpt_cnn/CNN_TEXT_"+str(epoch+1)+".pt")

    print('\n Epoch [{}/{}], Train Loss: {:.4f}, Validation Loss: {:.4f}, Validation Accuracy: {:.4f}, Validation F1: {:.4f}'.format(epoch+1, MAX_EPOCH, train_loss, val_loss, val_acc, val_f1))

# Epoch:   7%|▋         | 1/15 [01:03<14:48, 63.45s/it]
#  Epoch [1/15], Train Loss: 0.9828, Validation Loss: 1.0399, Validation Accuracy: 0.5085, Validation F1: 0.3738
# Epoch:  13%|█▎        | 2/15 [02:20<15:31, 71.66s/it]
#  Epoch [2/15], Train Loss: 0.9366, Validation Loss: 1.0460, Validation Accuracy: 0.5041, Validation F1: 0.3671
# Epoch:  20%|██        | 3/15 [03:44<15:25, 77.14s/it]
#  Epoch [3/15], Train Loss: 0.9391, Validation Loss: 1.0554, Validation Accuracy: 0.4956, Validation F1: 0.3646
# Epoch:  27%|██▋       | 4/15 [05:23<15:45, 85.93s/it]
#  Epoch [4/15], Train Loss: 0.9640, Validation Loss: 1.0540, Validation Accuracy: 0.4968, Validation F1: 0.3647
# Epoch:  33%|███▎      | 5/15 [07:09<15:29, 92.93s/it]
#  Epoch [5/15], Train Loss: 0.9582, Validation Loss: 1.0523, Validation Accuracy: 0.4993, Validation F1: 0.3674
# Epoch:  40%|████      | 6/15 [08:58<14:46, 98.51s/it]
#  Epoch [6/15], Train Loss: 0.9642, Validation Loss: 1.0565, Validation Accuracy: 0.4948, Validation F1: 0.3642
# Epoch:  47%|████▋     | 7/15 [10:46<13:32, 101.53s/it]
#  Epoch [7/15], Train Loss: 0.9660, Validation Loss: 1.0878, Validation Accuracy: 0.4637, Validation F1: 0.3239
# Epoch:  53%|█████▎    | 8/15 [12:35<12:07, 103.95s/it]
#  Epoch [8/15], Train Loss: 0.9774, Validation Loss: 1.0638, Validation Accuracy: 0.4875, Validation F1: 0.3552
# Epoch:  60%|██████    | 9/15 [14:25<10:35, 105.92s/it]
#  Epoch [9/15], Train Loss: 0.9780, Validation Loss: 1.0766, Validation Accuracy: 0.4749, Validation F1: 0.3356
# Epoch:  67%|██████▋   | 10/15 [16:15<08:54, 106.95s/it]
#  Epoch [10/15], Train Loss: 0.9709, Validation Loss: 1.0659, Validation Accuracy: 0.4856, Validation F1: 0.3563
# Epoch:  73%|███████▎  | 11/15 [18:04<07:10, 107.62s/it]
#  Epoch [11/15], Train Loss: 0.9655, Validation Loss: 1.0607, Validation Accuracy: 0.4906, Validation F1: 0.3608
# Epoch:  80%|████████  | 12/15 [19:52<05:23, 107.98s/it]
#  Epoch [12/15], Train Loss: 0.9754, Validation Loss: 1.0661, Validation Accuracy: 0.4854, Validation F1: 0.3537
# Epoch:  87%|████████▋ | 13/15 [21:41<03:36, 108.29s/it]
#  Epoch [13/15], Train Loss: 0.9840, Validation Loss: 1.0735, Validation Accuracy: 0.4778, Validation F1: 0.3515
# Epoch:  93%|█████████▎| 14/15 [23:32<01:48, 108.85s/it]
#  Epoch [14/15], Train Loss: 0.9954, Validation Loss: 1.0833, Validation Accuracy: 0.4681, Validation F1: 0.3401
# Epoch: 100%|██████████| 15/15 [25:23<00:00, 101.57s/it]
#  Epoch [15/15], Train Loss: 0.9905, Validation Loss: 1.0833, Validation Accuracy: 0.4681, Validation F1: 0.3437  


# No weights 
# kernel_sizes = [3,4,5] 
#  Epoch [8/15], Train Loss: 0.5756, Validation Loss: 0.9943, Validation Accuracy: 0.5319, Validation F1: 0.5041
# kernel_sizes = [4,5,6] 
# Epoch [12/15], Train Loss: 0.5570, Validation Loss: 0.9870, Validation Accuracy: 0.5425, Validation F1: 0.5204
# min_freq = 2
#  Epoch [9/15], Train Loss: 0.5744, Validation Loss: 0.9910, Validation Accuracy: 0.5388, Validation F1: 0.5167
# kernel_sizes = [2,3,4]
# Epoch [7/15], Train Loss: 0.6414, Validation Loss: 0.9785, Validation Accuracy: 0.5548, Validation F1: 0.5266

# CNN+BiLSTM
# Epoch [5/15], Train Loss: 0.2907, Validation Loss: 1.7040, Validation Accuracy: 0.5383, Validation F1: 0.5128

# with weights
# CNN 
# Epoch [14/15], Train Loss: 0.5678, Validation Loss: 0.9794, Validation Accuracy: 0.5550, Validation F1: 0.5352

Epoch:   7%|▋         | 1/15 [00:35<08:18, 35.64s/it]


 Epoch [1/15], Train Loss: 0.9379, Validation Loss: 0.9751, Validation Accuracy: 0.5516, Validation F1: 0.4065


Epoch:  13%|█▎        | 2/15 [01:07<07:14, 33.45s/it]


 Epoch [2/15], Train Loss: 0.8662, Validation Loss: 0.9735, Validation Accuracy: 0.5557, Validation F1: 0.4095


Epoch:  20%|██        | 3/15 [01:39<06:34, 32.87s/it]

...


 Epoch [13/15], Train Loss: 0.5697, Validation Loss: 0.9783, Validation Accuracy: 0.5539, Validation F1: 0.5352


Epoch:  93%|█████████▎| 14/15 [07:35<00:32, 32.56s/it]


 Epoch [14/15], Train Loss: 0.5678, Validation Loss: 0.9794, Validation Accuracy: 0.5550, Validation F1: 0.5352


Epoch: 100%|██████████| 15/15 [08:06<00:00, 32.43s/it]


 Epoch [15/15], Train Loss: 0.5667, Validation Loss: 0.9921, Validation Accuracy: 0.5386, Validation F1: 0.5266


In [ ]:
class BiLSTM(nn.Module):
    
    def __init__(self, args):
        super(BiLSTM, self).__init__()
        self.args = args
        self.hidden_dim = args.lstm_hidden_dim
        self.num_layers = args.lstm_num_layers
        V = args.embed_num
        D = args.embed_dim
        C = args.class_num
        # self.embed = nn.Embedding(V, D, max_norm=config.max_norm)
        self.embed = nn.Embedding(V, D, padding_idx=args.paddingId)
        # pretrained  embedding
        if args.word_Embedding:
            self.embed.weight.data.copy_(args.pretrained_weight)
        self.bilstm = nn.LSTM(D, self.hidden_dim // 2, num_layers=1, dropout=args.dropout, bidirectional=True, bias=False)
        print(self.bilstm)

        self.hidden2label1 = nn.Linear(self.hidden_dim, self.hidden_dim // 2)
        self.hidden2label2 = nn.Linear(self.hidden_dim // 2, C)
        # self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        embed = self.embed(x)
        x = embed.view(len(x), embed.size(1), -1)
        bilstm_out, _ = self.bilstm(x)

        bilstm_out = torch.transpose(bilstm_out, 0, 1)
        bilstm_out = torch.transpose(bilstm_out, 1, 2)
        bilstm_out = F.tanh(bilstm_out)
        bilstm_out = F.max_pool1d(bilstm_out, bilstm_out.size(2)).squeeze(2)
        y = self.hidden2label1(bilstm_out)
        y = self.hidden2label2(y)
        logit = y
        return logit

In [ ]:

vocabulary_size = len(TEXT.vocab.stoi) 
hidden_dim = 512
embedding_dim = 300 
num_layers = 2
dropout = 0.2

# The output size of labels.
output_size = 3

# learning rate is set to be 0.01.
lr = 0.001        

# The number of iteration is set to be 5.
num_epoch = 20 

class BiLSTMrgs:
  def __init__(self, hidden_dim, num_layers, vocabulary_size, embedding_dim, output_size, dropout) -> None:
      self.lstm_hidden_dim = hidden_dim
      self.lstm_num_layers = num_layers
      self.embed_num = vocabulary_size 
      self.embed_dim = embedding_dim
      self.class_num = output_size
      self.dropout = dropout
      self.paddingId = None
      self.word_Embedding = None
      self.cuda = True


args_for_lstm = BiLSTMrgs(hidden_dim, num_layers, vocabulary_size, embedding_dim, output_size, dropout)
model = BiLSTM(args_for_lstm).to(device)

import torch.optim as optim
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.1)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)   # define a optimizer for backpropagation
loss_func = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

 

criterion = nn.CrossEntropyLoss()

LSTM(300, 256, bias=False, dropout=0.2, bidirectional=True)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
# Train the model BiLSLM
MAX_EPOCH = 15
total_step = len(train_iter)
loss_list = []
acc_list = []

for epoch in trange(MAX_EPOCH, desc="Epoch"):
    train_loss = train(model, train_iter, optimizer, criterion)  
    val_loss, val_acc, val_f1 = evaluate(model, val_iter, criterion)

    # Create checkpoint at end of each epoch
    state_dict_model = model.state_dict() 
    state = {
        'epoch': epoch,
        'state_dict': state_dict_model,
        'optimizer': optimizer.state_dict()
        }

    # torch.save(state, "./drive/My Drive/Colab Notebooks/ckpt_cnn/CNN_TEXT_"+str(epoch+1)+".pt")

    print('\n Epoch [{}/{}], Train Loss: {:.4f}, Validation Loss: {:.4f}, Validation Accuracy: {:.4f}, Validation F1: {:.4f}'.format(epoch+1, MAX_EPOCH, train_loss, val_loss, val_acc, val_f1))
# Epoch [15/15], Train Loss: 0.0102, Validation Loss: 2.1585, Validation Accuracy: 0.5452, Validation F1: 0.5228

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
Epoch:   7%|▋         | 1/15 [01:57<27:24, 117.43s/it]


 Epoch [1/15], Train Loss: 0.8965, Validation Loss: 0.9958, Validation Accuracy: 0.5100, Validation F1: 0.5047


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
Epoch:  13%|█▎        | 2/15 [03:58<25:56, 119.72s/it]


 Epoch [2/15], Train Loss: 0.5663, Validation Loss: 1.0299, Validation Accuracy: 0.5500, Validation F1: 0.5221


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
Epoch:  20%|██        | 3/15 [05:57<23:49, 119.10s/it]

...


 Epoch [13/15], Train Loss: 0.0118, Validation Loss: 2.1103, Validation Accuracy: 0.5447, Validation F1: 0.5181


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
Epoch:  93%|█████████▎| 14/15 [27:38<01:54, 114.19s/it]


 Epoch [14/15], Train Loss: 0.0098, Validation Loss: 2.2797, Validation Accuracy: 0.5496, Validation F1: 0.5066


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
Epoch: 100%|██████████| 15/15 [29:33<00:00, 118.22s/it]


 Epoch [15/15], Train Loss: 0.0102, Validation Loss: 2.1585, Validation Accuracy: 0.5452, Validation F1: 0.5228


In [ ]:
class CNN_Text_emb(nn.Module):
    def __init__(self, vocabulary_size, embedding_dim, output_size, kernel_num, region_sizes, dropout):
        '''
        vocabulary_size: vocabulary size
        embedding_dim: word embedding size
        output_size: number of classes in prediction
        kernel_num: number of kernels (number of output channels of convolutional layers)
        region_sizes: height of kernels of convolutional layers
        dropout: dropout rate
        '''
        super(CNN_Text_emb, self).__init__()
        Ci = 1
        self.embeddings = nn.Embedding.from_pretrained(pre_trained_emb)
        self.convolution_layers = nn.ModuleList([nn.Conv2d(in_channels = Ci, out_channels = kernel_num, kernel_size = (K, embedding_dim)) for K in region_sizes])
        self.dropout = nn.Dropout(dropout) 
        self.fc = nn.Linear(len(kernel_sizes) * kernel_num, output_size)

    def forward(self, x):        
        input_embeddings = self.embeddings(x)  
        input_embeddings = input_embeddings.permute(1,0,2)
        input_embeddings = input_embeddings.unsqueeze(1)
        convolute_outputs = [F.relu(conv(input_embeddings)).squeeze(3) for conv in self.convolution_layers]  
        max_pooling_outputs = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in convolute_outputs] 
        concat_list = torch.cat(max_pooling_outputs, 1) 
        drop_output = self.dropout(concat_list) 
        fc1_output = self.fc(drop_output) 
        final_output = F.softmax(fc1_output,dim=1)
        return final_output

In [ ]:
class CNN_Text(nn.Module):
    def __init__(self, vocabulary_size, embedding_dim, output_size, kernel_num, region_sizes, dropout):
        '''
        vocabulary_size: vocabulary size
        embedding_dim: word embedding size
        output_size: number of classes in prediction
        kernel_num: number of kernels (number of output channels of convolutional layers)
        region_sizes: height of kernels of convolutional layers
        dropout: dropout rate
        '''
        super(CNN_Text, self).__init__()
        # the size of input channel is 1.
        Ci = 1
        
        # word embedding layer
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size, embedding_dim = embedding_dim )
        
        # convolution with kernels
        self.convolution_layers = nn.ModuleList([nn.Conv2d(in_channels = Ci, out_channels = kernel_num, kernel_size = (K, embedding_dim)) for K in region_sizes])
        
        # a dropout layer
        self.dropout = nn.Dropout(dropout) 
        
        # fully connected layer
        self.fc = nn.Linear(len(kernel_sizes) * kernel_num, output_size)

    def forward(self, x):
        # input x  [sequence length, batch size]
        
        input_embeddings = self.embeddings(x)  
        # (batch size, word_sequence, embedding_dim) word embedding

        input_embeddings = input_embeddings.permute(1,0,2)
        input_embeddings = input_embeddings.unsqueeze(1)
        #  [batch size, number of channel is one, sequence length, embeeding size]

        # convolutional layers
        convolute_outputs = [F.relu(conv(input_embeddings)).squeeze(3) for conv in self.convolution_layers]  
        
        # to get the maximum value of filtered tensor
        max_pooling_outputs = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in convolute_outputs] 
        
        concat_list = torch.cat(max_pooling_outputs, 1) # concatenate representations
        
        drop_output = self.dropout(concat_list)  # add drop layer
        
        fc1_output = self.fc(drop_output)  # get the fc1 using a fully connected layer
        
        final_output = F.softmax(fc1_output,dim=1)
        
        return final_output

In [ ]:
# Hyper Parameters

# the vocabulary size
vocabulary_size = len(TEXT.vocab.stoi) 

# Dimension of word embedding is 300. Namely, each word is expressed by a vector that has 300 dimensions.
embedding_dim = 400 

# region size as 2, 3, and 4
kernel_sizes = [3,2,1] 

# the number of kernel in each region size
kernels_num = 32 

# The dropout rate is set to be 0.5.
dropout = 0.2

# The output size of labels.
output_size = 3

# learning rate is set to be 0.01.
lr = 0.001        

# The number of iteration is set to be 5.
num_epoch = 20  

# employ class CNN_Text and assign to cnn
model = CNN_Text(vocabulary_size, embedding_dim, output_size, kernels_num, kernel_sizes, dropout).to(device)

In [ ]:
# Train the model DEEP CNN
MAX_EPOCH = 20
total_step = len(train_iter)
loss_list = []
acc_list = []

import torch.optim as optim
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.1)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)   # define a optimizer for backpropagation
loss_func = nn.CrossEntropyLoss()

for epoch in trange(MAX_EPOCH, desc="Epoch"):
    train_loss = train(model, train_iter, optimizer, criterion)  
    val_loss, val_acc, val_f1 = evaluate(model, val_iter, criterion)

    # Create checkpoint at end of each epoch
    state_dict_model = model.state_dict() 
    state = {
        'epoch': epoch,
        'state_dict': state_dict_model,
        'optimizer': optimizer.state_dict()
        }

    # torch.save(state, "./drive/My Drive/Colab Notebooks/ckpt_cnn/CNN_TEXT_"+str(epoch+1)+".pt")

    print('\n Epoch [{}/{}], Train Loss: {:.4f}, Validation Loss: {:.4f}, Validation Accuracy: {:.4f}, Validation F1: {:.4f}'.format(epoch+1, MAX_EPOCH, train_loss, val_loss, val_acc, val_f1))

Epoch:   5%|▌         | 1/20 [01:32<29:20, 92.68s/it]


 Epoch [1/20], Train Loss: 0.9758, Validation Loss: 0.9931, Validation Accuracy: 0.5254, Validation F1: 0.3860


Epoch:  10%|█         | 2/20 [03:01<27:05, 90.33s/it]


 Epoch [2/20], Train Loss: 0.8735, Validation Loss: 0.9853, Validation Accuracy: 0.5434, Validation F1: 0.4003


Epoch:  15%|█▌        | 3/20 [04:28<25:13, 89.04s/it]

...


 Epoch [18/20], Train Loss: 0.5685, Validation Loss: 0.9924, Validation Accuracy: 0.5404, Validation F1: 0.5105


Epoch:  95%|█████████▌| 19/20 [27:57<01:27, 87.27s/it]


 Epoch [19/20], Train Loss: 0.5675, Validation Loss: 0.9932, Validation Accuracy: 0.5415, Validation F1: 0.5197


Epoch: 100%|██████████| 20/20 [29:23<00:00, 88.16s/it]


 Epoch [20/20], Train Loss: 0.5671, Validation Loss: 0.9929, Validation Accuracy: 0.5383, Validation F1: 0.5064


In [ ]:
import scipy.stats
import random
import torch.optim as optim
LEARNING_RATE=.1
MAX_EPOCH=20


def random_search(num_iter):
    results = []
    for i in range(num_iter):
        config = {
            #define hyperparameters here
            "kernel_sizes": random.choice([[1,2,3],[2,3,4],[3,4,5],[4,5,6],[1,4,6],[1,3,6],[2,5,4]]),
            "embedding_dim": random.choice([300, 400, 500, 600, 700, 800]),
            "dropout" : random.choice([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]),
            "lr": random.choice([0.1, 0.01, 0.001, 0.0001, 0.00001])

            
        }
        
        print("new config")
        print(config)
        model = CNN_Text(vocabulary_size=len(TEXT.vocab.stoi), embedding_dim=config["embedding_dim"], output_size=3, kernel_num=32, region_sizes=config["kernel_sizes"],  dropout=config["dropout"]).to(device)
        # model = ConvNet_1(layer_num=config["layers"],filtersize=config["kernel_size"],filters=config["n_filters"],
        #                 nonlin=config["activation"], output_size=5, VOCAB_SIZE=VOCAB_SIZE, WORD_VEC_SIZE=config["WORD_VEC_SIZE"],
        #                 num_layers=config["num_layers"], dropout=config["dropout"])

    #     class CNN_Text(nn.Module):
    # def __init__(self, vocabulary_size, embedding_dim, output_size, kernel_num, region_sizes, dropout):
    #     '''

        import torch.optim as optim
        def init_weights(m):
            for name, param in m.named_parameters():
                if 'weight' in name:
                    nn.init.normal_(param.data, mean=0, std=0.1)
                else:
                    nn.init.constant_(param.data, 0)
            
        model.apply(init_weights)
        optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])   # define a optimizer for backpropagation
        loss_func = nn.CrossEntropyLoss()
        model.to(device)
 
    
        max_val = 0
        best_epoch = 0
        for epoch in range(20):
        # train the model for one pass over the data
            train_loss = train(model, train_iter, optimizer, criterion) 
        # compute the training accuracy
        # compute the validation accuracy
            val_loss, val_acc, val_f1 = evaluate(model, val_iter, criterion)
            print(val_acc)
            if val_acc > max_val:
                max_val = val_acc
                best_epoch = epoch+1
        # print the loss for every epoch
            print('\n Epoch [{}/{}], Train Loss: {:.4f}, Validation Loss: {:.4f}, Validation Accuracy: {:.4f}, Validation F1: {:.4f}'.format(epoch+1, MAX_EPOCH, train_loss, val_loss, val_acc, val_f1))
        results.append((max_val,best_epoch,config))
    return results

In [ ]:
random_search(30)

new config
{'kernel_sizes': [2, 5, 4], 'embedding_dim': 600, 'dropout': 0.7, 'lr': 1e-05}
0.4266903914590747

 Epoch [1/20], Train Loss: 1.1106, Validation Loss: 1.0838, Validation Accuracy: 0.4267, Validation F1: 0.1994
0.4266903914590747

 Epoch [2/20], Train Loss: 1.1051, Validation Loss: 1.0791, Validation Accuracy: 0.4267, Validation F1: 0.1994
0.4266903914590747

 Epoch [3/20], Train Loss: 1.0927, Validation Loss: 1.0743, Validation Accuracy: 0.4267, Validation F1: 0.1994
0.4268683274021352

 Epoch [4/20], Train Loss: 1.0886, Validation Loss: 1.0694, Validation Accuracy: 0.4269, Validation F1: 0.1997
0.42758007117437724

 Epoch [5/20], Train Loss: 1.0845, Validation Loss: 1.0641, Validation Accuracy: 0.4276, Validation F1: 0.2011
0.4295373665480427

 Epoch [6/20], Train Loss: 1.0803, Validation Loss: 1.0596, Validation Accuracy: 0.4295, Validation F1: 0.2055
0.4318505338078292

 Epoch [7/20], Train Loss: 1.0710, Validation Loss: 1.0548, Validation Accuracy: 0.4319, Validation F1:

In [ ]:
# {'kernel_sizes': [1, 4, 6], 'embedding_dim': 400, 'dropout': 0.2, 'lr': 0.001}
